In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
import os
import time
from selenium.webdriver.common.by import By
import re
from timeit import default_timer as timer
from difflib import SequenceMatcher
import pandas as pd

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

options = Options()
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--headless")
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--no-sandbox")  # Bypass OS security model
options.add_argument("--disable-dev-shm-usage")  # Overcome resource problems
options.add_argument("--disable-gpu")  # Disable GPU
options.add_argument("--window-size=400,400")  # Set a fixed window size (optional but helps)
options.add_argument("--disable-software-rasterizer")  # Disable rendering components
options.add_argument("--hide-scrollbars")  # Hide scrollbars
print("Done")

Done


In [4]:
def search(card, options):
    driver = webdriver.Chrome(options = options)
    driver.get("https://tagger.scryfall.com/")
    search_box = driver.find_element(By.CLASS_NAME, "tag-input-field")
    search_box.send_keys(card)
    time.sleep(2)

    selecter = driver.find_element(By.CLASS_NAME, "suggestion-menu") #Finds the dropdown of suggestions
    time.sleep(2)

    search = selecter.find_elements(By.CLASS_NAME, "suggestion-row") #Pulls all of the rows of the dropdown
    searchlist = []
    for i in search:
        searchlist.append(i.get_attribute("innerHTML")) 
    cardstrings = []
    similarlist = []
 
    for i in searchlist: 
        if card in i:
            if "#icons-card" in i: #Find the value of the dropdown that is a specific card
                i = i.split(">")
                for j in i:
                    if card in j:
                        cardstrings.append(j.replace(" <!----",""))
    
    for i in cardstrings:
        sim = similar(card, i)
        similarlist.append(sim)

    
    cardsearch = similarlist.index(max(similarlist))

    if "#icons-tag" in searchlist[cardsearch+1]:
        selecter.find_elements(By.CLASS_NAME, "suggestion-row")[cardsearch + 2].click()
    else:
        selecter.find_elements(By.CLASS_NAME, "suggestion-row")[cardsearch + 1].click()
    time.sleep(1)


    try:
        specific = driver.find_element(By.CLASS_NAME, "card-grid-item")
        specific.click()
    except:
        pass

    time.sleep(2)
    tags = driver.find_elements(By.CLASS_NAME, "tag-row-flex")

    taglist = []
    for i in tags:
        taglist.append(i.get_attribute("innerHTML"))

    cardtags = []
    for i in taglist:
        if "tags/card/" in i:
            cardtags.append(i)
    
    tagsavelist = []
    for i in cardtags:
        if "tags/card/" in i:
            tag = i.split('"">')[-1].replace("</a>","")
            tagsavelist.append(tag)

    time.sleep(2)
    oldtags = driver.find_elements(By.CLASS_NAME, "tagging-ancestors")

    taglist = []
    for i in oldtags:
        taglist.append(i.get_attribute("innerHTML"))

    cardtags = []
    for i in taglist:
        if "tags/card/" in i:
            cardtags.append(i)

    for i in cardtags:
        if "/tags/card/" in i:
            tag = i.split(",")
            for k in tag:
                tagsavelist.append(k.split('"">')[-1].replace("</a>",""))
            

    return tagsavelist
    

card = "Kianne, Dean of Substance // Imbraham, Dean of Theory"
searchlist = search(card, options)
searchlist

['cycle-stx-dean',
 'evasion',
 'gives pp counters',
 'mana sink',
 'repeatable token generator',
 'cycle']

In [5]:
dataset = pd.read_csv("updated-cardlist.csv")
if "Unnamed: 0" in dataset.columns:
    dataset.drop("Unnamed: 0", axis=1, inplace=True)
totalcards = dataset["cardname"]
tagdict = {}
errorlist = []

In [ ]:
count = 0
done = pd.read_csv("scryfall_taglist.csv")
if "Unnamed: 0" in done.columns:
    done.drop("Unnamed: 0", axis=1, inplace=True)
for i in totalcards:
    if i in list(done["Card"]):
        continue
    else:
        try:
            tagdict[i] = search(i, options)
        except:
            tagdict[i] = []
            errorlist.append(i)
            print("----- Error in finding tags for", i)
            continue
        
        if count % 10 == 0:
            print(i)

        if count % 100 == 0:
            newdf = pd.DataFrame(list(tagdict.items()), columns=['Card', 'Tags'])
            df = pd.concat([done, newdf], axis=0)
            df.reset_index(inplace = True)
            if "index" in df.columns:
                df.drop("index", axis = 1, inplace = True)
            df.to_csv("scryfall_taglist.csv")
            print("---- Saved to csv ----")
        count += 1


Bosh, Iron Golem
---- Saved to csv ----
Cobalt Golem
Deconstruct
Electrostatic Bolt
Fists of the Anvil
Goblin Charbelcher
Grid Monitor
Isochron Scepter
Leonin Elder
Lodestone Myr
Malachite Golem
---- Saved to csv ----
Molder Slug
Necrogen Mists
Nim Shambler
Pewter Golem
Quicksilver Elemental
Rustmouth Ogre
Shared Fate
Slith Predator
Spikeshot Goblin
Talisman of Impulse
---- Saved to csv ----
Tempest of Light
Tower of Eons
Vermiculos
Wall of Blood
Aether Vial
Arcbound Ravager
Carry Away
Darksteel Gargoyle
Drooling Ogre
Essence Drain
---- Saved to csv ----
Hallow
Leonin Bola
Murderous Spoils
Oxidda Golem
Purge
Scavenging Scarab
Soulscour
Sword of Fire and Ice
Test of Faith
Viridian Zealot
---- Saved to csv ----
Acquire
Auriok Windwalker
Blind Creeper
Clearwater Goblet
Devour in Shadow
Ensouled Scimitar
Fist of Suns
Guardian Idol
Joiner Adept
Magma Jet
---- Saved to csv ----
Nim Grotesque
Raksha Golden Cub
Sawtooth Thresher
Spark Elemental
Suncrusher
Trinket Mage
Wayfarer's Bauble
Battle-

In [18]:
a = {}
a["test"] = [1]
a["test2"] = [2]
df = pd.DataFrame(list(tagdict.items()), columns=['Card', 'Tags'])

In [19]:
df

,Card,Tags
0,Air Elemental,"[evasion, french vanilla, flavors of vanilla]"
1,Animate Artifact,"[alliteration, animate artifact, mana value ma..."


In [16]:
1 % 10

1

In [3]:
done = pd.read_csv("scryfall_taglist.csv")

In [8]:
done["Card"]

0         Air Elemental
1      Animate Artifact
2          Animate Dead
3          Animate Wall
4        Ankh of Mishra
             ...       
100          Holy Armor
101       Holy Strength
102    Howl from Beyond
103        Howling Mine
104    Hurloon Minotaur
Name: Card, Length: 105, dtype: object

In [12]:
"Animate Artifact" in list(done["Card"])

True

In [2]:
import pandas as pd
done = pd.read_csv("scryfall_taglist.csv")
done

,Unnamed: 0,Card,Tags
0,0,Hurricane,"['activate from hand', 'hate-flying', 'hurrica..."
1,1,Air Elemental,"['evasion', 'french vanilla', 'flavors of vani..."
2,2,Animate Artifact,"['alliteration', 'animate artifact', 'mana val..."
3,3,Animate Dead,"['enchant change', 'meme', 'reanimate-creature..."
4,4,Animate Wall,"['tribal-wall', 'turns off defender', 'unique ..."
...,...,...,...
6215,1196,Blinding Beam,"['alliteration', 'freeze-creature', 'modal', '..."
6216,1197,Blinkmoth Urn,"['adds multiple mana', 'ramp', 'symmetrical', ..."
6217,1198,Blinkmoth Well,"['hate-artifact', 'hate-set-mechanic', 'tapper..."
6218,1199,Bloodscent,"['lure', 'combat manipulation', 'force blocker']"
